In [47]:
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
import json

In [48]:
#pegando os dados que precisamos

url = "https://www.nba.com/stats/players/traditional?PerMode=Totals&sort=PTS&dir=-1"


top10ranking = {}

rankings = {
    '3points': {'field': 'FG3M', 'label': '3PM'},
    'points': {'field': 'PTS', 'label': 'PTS'},
    'assistants': {'field': 'AST', 'label': 'AST'},
    'rebounds': {'field': 'REB', 'label': 'REB'},
    'steals': {'field': 'STL', 'label': 'STL'},
    'blocks': {'field': 'BLK', 'label': 'BLK'},
}
def buildrank(type):

    field = rankings[type]['field'] 
    label = rankings[type]['label'] 
    
    driver.find_element(By.XPATH, f"//div[@class='Crom_container__C45Ti crom-container']//table//thead//tr//th[@field='{field}']").click
    element = driver.find_element(By.XPATH, "//div[@class='Crom_container__C45Ti crom-container']//table")
    html_content = element.get_attribute('outerHTML')
    
    soup = BeautifulSoup(html_content, 'html5lib')
    table = soup.find(name='table')
    
    df_full = pd.read_html(str(table))[0].head(10)
    df = df_full[['Unnamed: 0', 'Player', 'Team', label]]
    df.columns = ['pos', 'player', 'team', 'total']
   
    return  df.to_dict('records')

In [49]:
options = Options()
options.headless = True
driver = webdriver.Firefox(options=options)

driver.get(url)
time.sleep(15)

#essa linha aqui ta sendo usada pra poder aceitar os cookies automaticamente
driver.find_element(By.XPATH,"//div[@class='ot-sdk-container']//div[@class='ot-sdk-row']//div[@class='ot-sdk-three ot-sdk-columns has-reject-all-button']//div[@id='onetrust-button-group']//button[@id='onetrust-accept-btn-handler']").click()
time.sleep(5)

for k in rankings:
    top10ranking[k] = buildrank(k)
    
driver.quit()

In [50]:
js = json.dumps(top10ranking)
fp = open('ranking.json', 'w')
fp.write(js)
fp.close()